# Histowheel

Histogram wheel in Bokeh

In [1]:
%pylab inline

import numpy as np

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid, Range1d
from bokeh.models.glyphs import AnnularWedge, Text

Populating the interactive namespace from numpy and matplotlib


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def histowheel(data, colors_24_7=None, inner_radius=4):

    largest_bin_size = max(data)
    
    angle = 2 * np.pi / (7 * 24)
    start_angle = np.array([np.pi/2 - angle*i for i in range(len(data))])

    plot = Plot(
        plot_width=800,
        plot_height=800,
    )

    # plot a bar (or wedge) for each day
    source = ColumnDataSource({
        'inner_radius': np.zeros(data.shape[0]) + inner_radius,
        'outer_radius': data + inner_radius,
        'start_angle': start_angle,
        'end_angle': start_angle - angle,
        'color': colors_24_7 or ['#8888ee']*24*7,
    })
    glyph = AnnularWedge(
        x=0, y=0, 
        inner_radius='inner_radius', 
        outer_radius='outer_radius', 
        start_angle='start_angle', 
        end_angle='end_angle', 
        fill_alpha=.8,
        fill_color='color',
        line_alpha=0.5,
        line_color='black',
        direction='clock'
    )
    plot.add_glyph(source, glyph)

    # add labels for the days of the week
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    week_angle = 2 * np.pi / 7
    week_angles = np.array([-i*week_angle + np.pi/2 for i in range(7)])
    week_angles_midday = week_angles - np.pi/7
    week_source = ColumnDataSource({
        'x': .75*inner_radius * np.cos(week_angles_midday),
        'y': .75*inner_radius * np.sin(week_angles_midday),
        'text': day_names
    })
    t = Text(x='x', y='y', text='text', text_align='center', text_baseline='middle')
    plot.add_glyph(week_source, t)
    
    # add edges to separate the days of the week
    source = ColumnDataSource({
        'angle': week_angles
    })
    glyph = AnnularWedge(
        x=0, 
        y=0, 
        inner_radius=inner_radius, 
        outer_radius=inner_radius+largest_bin_size, 
        start_angle='angle', 
        end_angle='angle', 
    )
    plot.add_glyph(source, glyph)
    
    margin_coef = 1.2
    half_range = margin_coef * (largest_bin_size + inner_radius)
    plot.x_range = Range1d(-half_range, half_range)
    plot.y_range = Range1d(-half_range, half_range)

    return plot

In [4]:
data = [
    3, 3, 2, 2, 4, 4, 4, 4, 5, 5, 5, 2, 2, 3, 2, 0, 2, 2, 3, 2, 1, 0, 1, 1,
    2, 3, 2, 2, 4, 4, 4, 4, 5, 5, 5, 2, 2, 3, 2, 0, 0, 0, 3, 2, 1, 0, 0, 2,
    2, 3, 2, 2, 4, 4, 4, 4, 5, 5, 5, 2, 2, 3, 1, 1, 1, 2, 3, 2, 1, 0, 0, 1,
    2, 3, 2, 2, 4, 4, 4, 6, 6, 6, 5, 2, 2, 3, 2, 0, 2, 2, 3, 2, 1, 0, 1, 1,
    2, 3, 2, 2, 4, 4, 4, 4, 5, 5, 5, 4, 4, 5, 6, 4, 2, 2, 3, 2, 1, 0, 2, 4,
    2, 3, 2, 2, 4, 4, 4, 4, 5, 5, 5, 2, 2, 3, 2, 0, 2, 2, 3, 2, 1, 0, 2, 1,
    0, 1, 2, 3, 4, 5, 6, 5, 6, 5, 6, 5, 6, 5, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1,
]
data = np.array(data)

color_map = {
    'd': '#eeee88',  # weekday
    'e': '#ee8888',  # weekend
    'n': '#8888ee',  # night
    't': '#88ee88'  # twilight
}

# starting Monday 12am, one character per hour
week = (
    'n'*6 + 't'*2 + 'd'*8 + 't'*6 + 'n'*2 +  # Monday
    'n'*6 + 't'*2 + 'd'*8 + 't'*6 + 'n'*2 +  # Tuesday
    'n'*6 + 't'*2 + 'd'*8 + 't'*6 + 'n'*2 +  # Wednesday
    'n'*6 + 't'*2 + 'd'*8 + 't'*6 + 'n'*2 +  # Thursday
    'n'*6 + 't'*2 + 'd'*8 + 't'*6 + 'n'*2 +  # Friday
    'n'*6 + 't'*2 + 'e'*8 + 't'*6 + 'n'*2 +  # Saturday
    'n'*6 + 't'*2 + 'e'*8 + 't'*6 + 'n'*2  # Sunday
)
colors = [color_map[symb] for symb in week]

plot = histowheel(data, colors, inner_radius=3)
show(plot)